In [1]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails
from azure.identity import DefaultAzureCredential
import jsonref
from dotenv import load_dotenv

load_dotenv()
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_GPT4o_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_AISTUDIO_PROJECT_CONN_STRING = os.getenv("AZURE_AISTUDIO_PROJECT_CONN_STRING")

In [2]:
# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), 
    conn_str=AZURE_AISTUDIO_PROJECT_CONN_STRING
)

In [3]:
with open('./OpenAPI specs/weather_openapi.json', 'r') as f:
    openapi_spec = jsonref.loads(f.read())

# Create Auth object for the OpenApiTool (note that connection or managed identity auth setup requires additional setup in Azure)
auth = OpenApiAnonymousAuthDetails()

# Initialize agent OpenAPI tool using the read in OpenAPI spec
openapi = OpenApiTool(name="get_weather", spec=openapi_spec, description="Retrieve weather information for a location", auth=auth)

In [4]:
# Create agent with OpenAPI tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="my-assistant",
        instructions="You are a helpful assistant",
        tools=openapi.definitions
    )
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")
    
    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="What's the weather in Seattle?",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

Created agent, ID: asst_P2dJvgkdldzmo4LGlh6n2HHK
Created thread, ID: thread_8fVGrnz7SsbmNKXkqEbjSg9W
Created message, ID: msg_fKoYYlVg8q8C9td6Lla0Wn8c
Run finished with status: completed
Deleted agent
Messages: {'object': 'list', 'data': [{'id': 'msg_G4uOQETO9ByxQ2FJVCB7llTK', 'object': 'thread.message', 'created_at': 1755168687, 'assistant_id': 'asst_P2dJvgkdldzmo4LGlh6n2HHK', 'thread_id': 'thread_8fVGrnz7SsbmNKXkqEbjSg9W', 'run_id': 'run_brKJOqyknxL8zwKuIz2Htf84', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The current weather in Seattle is:\n\n- **Condition:** Overcast  \n- **Temperature:** 23°C (73°F)  \n- **Feels Like:** 25°C (77°F)  \n- **Humidity:** 59%  \n- **Visibility:** 16 km (9 miles)  \n- **Wind:** 4 km/h (2 mph) from the Southwest  \n- **Precipitation:** None  \n\nLet me know if you need more details! 🌥️', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_fKoYYlVg8q8C9td6Lla0Wn8c', 'object': 'thread.message', 'created_at': 1755